In [1]:
# Import libraries
import pandas as pd
import yfinance as yf
import datetime as dt
import numpy as np
import requests as re

### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Steps:
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame.
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**.
2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Holdings” → `Holdings`
   - “Ltd” or “Limited” → `Ltd`
   - Others → `Other`

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.

3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`
4. Convert `Shares Offered` to numeric, clean missing or invalid values.
5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)
6. Group by `Company Class` and calculate total withdrawn value.
7. **Answer**: Which class had the highest **total** value of withdrawals?

In [2]:
# import requests library
import requests as re

In [3]:
# import data. Use the user agent to bypass the 403 status error
response = re.get('https://stockanalysis.com/ipos/withdrawn',headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'
})

ipo_content = pd.read_html(response.text)
ipo_df = ipo_content[0]


/tmp/ipython-input-3-1717818236.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_content = pd.read_html(response.text)


In [4]:
ipo_df.head(4)

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-


In [5]:
# Add a column of company class
ipo_df['Company Class']=ipo_df['Company Name'].apply(lambda x:x.split(' ')[-1].replace(r'.','') if isinstance(x,str) else None)
ipo_df['Company Class'] = ipo_df['Company Class'].str.replace('Limited','Ltd')
ipo_df['Company Class'] = np.where(ipo_df['Company Class'].str.contains(r'(Group|Acq\.Corp|Ltd|Limited|Inc)'),ipo_df['Company Class'],"Others")

/tmp/ipython-input-5-3607541126.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ipo_df['Company Class'] = np.where(ipo_df['Company Class'].str.contains(r'(Group|Acq\.Corp|Ltd|Limited|Inc)'),ipo_df['Company Class'],"Others")


In [6]:
### Calculate average prices
# Define a custom function
def transPrice(a):
    if(len(a)!=1):
        st = a.replace('$','')
        stl = st.split(' - ')
        stl = [float(i) for i in stl]
        return np.mean(stl)
    else:
        return None

def transShare(a):
    try:
        return float(a)
    except(ValueError):
        return 0

# Add an Avg. price column
ipo_df = ipo_df.assign(**{'Avg Price': ipo_df['Price Range'].apply(transPrice)},
                    **{'Shares Offered': ipo_df['Shares Offered'].apply(transShare)})

# total withdrawn values by company class

ipo_df['Withdrawn Value'] = ipo_df['Avg Price']*ipo_df['Shares Offered']



In [7]:
ipo_df.groupby('Company Class').agg({'Withdrawn Value': 'sum'}).sort_values('Withdrawn Value',ascending=False)

,Withdrawn Value
Company Class,
Others,4.688920e+09
Inc,2.257164e+09
Ltd,7.313346e+08
Group,2.718750e+07


---
### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.


#### Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
   [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
   Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
   ➤ You should have **75 tickers**.

2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
   Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.


3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

   ```python
   stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
4. Filter the DataFrame to keep data only for the trading day:  
   **‘2025-06-06’**

   Compute descriptive statistics (e.g., `.describe()`) for these columns:  
   - `growth_252d`  
   - `Sharpe`

   You should observe:  
   - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
   - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

5. **Answer:**  
   - What is the **median Sharpe ratio** for these 71 stocks?  
   - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
   - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`?

In [8]:
ipo_response = re.get('https://stockanalysis.com/ipos/2024/',headers= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'})
ipo_24 = pd.read_html(ipo_response.text)[0]
ipo_24['IPO Date'] = pd.to_datetime(ipo_24['IPO Date'])
ipo_24_filtered = ipo_24[(ipo_24['IPO Date']<'2024-06-01') & (ipo_24['IPO Price']!='-')]
ipo_24_filtered.shape[0]

/tmp/ipython-input-8-3828414514.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_24 = pd.read_html(ipo_response.text)[0]


75

In [ ]:
ipo_24['Symbol'].to_list()

In [10]:
tickerl = ipo_24_filtered['Symbol'].to_list()
ticker_history =[]
for ticker in tickerl:
  df = yf.download(ticker,period='max',interval='1d',multi_level_index=False)[['Close','Volume']].sort_index()
  df['Date'] = df.index
  df['Ticker'] = ticker
  df = df.reset_index(drop=True)
  df['growth_30day'] = df['Close']/df['Close'].shift(30)
  df['growth_252day'] = df['Close']/df['Close'].shift(252)

  # annualise standard deviation. 252 trading days
  df['volatility'] =   df['Close'].rolling(30).std()*np.sqrt(252)

  # 4.5% is a risk-free rate
  df['Sharpe Ratio'] = (df['growth_252day'] - 0.045)/df['volatility']
  ticker_history.append(df[df['Date']=='2025-06-06'])

/tmp/ipython-input-10-3568685496.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker,period='max',interval='1d',multi_level_index=False)[['Close','Volume']].sort_index()
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-10-3568685496.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker,period='max',interval='1d',multi_level_index=False)[['Close','Volume']].sort_index()
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-10-3568685496.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker,period='max',interval='1d',multi_level_index=False)[['Close','Volume']].sort_index()
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-10-3568685496.py:4: FutureWarning: YF.download() has changed argument auto_adjust default 

In [11]:
ticker_history_df = pd.concat(ticker_history,axis=0).reset_index(drop=True)
ticker_history_df = ticker_history_df[ticker_history_df['growth_252day'].isna()==False]
ticker_history_df.head()

,Close,Volume,Date,Ticker,growth_30day,growth_252day,volatility,Sharpe Ratio
0,36.389999,137900,2025-06-06,BOW,0.879623,1.442331,24.508713,0.057014
1,20.410000,1300,2025-06-06,HDL,0.969366,1.007155,18.816306,0.051134
3,0.295000,3428900,2025-06-06,JDZG,0.564054,0.168571,1.475159,0.083768
4,1.255000,501000,2025-06-06,RAY,0.459707,0.343459,9.314924,0.032041
5,1.290000,21500,2025-06-06,BTOC,1.084034,0.261663,3.101715,0.069853


In [12]:
ticker_history_df[['growth_252day','Sharpe Ratio']].describe()

,growth_252day,Sharpe Ratio
count,71.000000,71.000000
mean,1.152898,0.288285
std,1.406017,0.519028
min,0.024970,-0.079677
25%,0.293422,0.041215
50%,0.758065,0.083768
75%,1.362736,0.311507
max,8.097413,2.835668


---
### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

#### Steps:

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  

5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).


In [15]:
# prompt: create a list from 1 to 12

month_list = list(range(1, 13))

In [ ]:
tickerl = ipo_24_filtered['Symbol'].to_list()
ipo_history =[]
for ticker in tickerl:
  df = yf.download(ticker,period='max',interval='1d',multi_level_index=False)[['Close','Volume']].sort_index()
  df['Date'] = df.index
  df['Ticker'] = ticker
  for month in month_list:
    df[f'future_growth_{str(month)}m'] = df['Close'].shift(-month*21)/df['Close']

  df['min_date'] = df['Date'].min()
  df = df.reset_index(drop=True)
  ipo_history.append(df[df['Date']==df['Date'].min()])

In [30]:
ipo_history_df = pd.concat(ipo_history,axis=0).reset_index(drop=True)
ipo_history_df.shape

(75, 17)

In [32]:
ipo_history_df.describe()

,Close,Volume,Date,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m,min_date
count,75.000000,7.500000e+01,75,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000,74.000000,71.000000,75
mean,15.856623,5.341061e+06,2024-03-24 07:40:48,0.927259,0.940544,0.833825,0.825087,0.803769,0.864186,0.847150,0.832984,0.881780,0.917946,0.882535,0.900862,2024-03-24 07:40:48
min,0.011600,6.000000e+02,2024-01-09 00:00:00,0.098947,0.073800,0.060947,0.045368,0.054109,0.061432,0.044086,0.043103,0.033144,0.037769,0.023674,0.038947,2024-01-09 00:00:00
25%,4.140000,7.332000e+05,2024-02-08 12:00:00,0.778984,0.685815,0.511212,0.517233,0.448403,0.384560,0.296870,0.208677,0.226740,0.242424,0.264661,0.229211,2024-02-08 12:00:00
50%,10.040000,1.992600e+06,2024-03-28 00:00:00,0.977000,1.000000,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.772592,0.717585,0.634667,2024-03-28 00:00:00
75%,19.900000,5.451900e+06,2024-04-25 00:00:00,1.046509,1.154013,1.069085,1.134300,1.016381,1.093948,1.114468,1.082457,1.049719,1.200830,1.106780,1.136392,2024-04-25 00:00:00
max,98.000000,4.870550e+07,2024-09-11 00:00:00,2.646505,4.874759,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,4.849711,2024-09-11 00:00:00
std,18.235333,8.882742e+06,NaN,0.346261,0.574545,0.409763,0.401969,0.488349,0.653180,0.712944,0.762423,0.936957,0.911431,0.863183,0.892906,NaN


### Question 4: [Strategy] Simple RSI-Based Trading Strategy


**What is the total profit (in $thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**


---

#### Goal:
Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.

---


#### Steps:

1. **Run the full notebook from Lecture 2 (33 stocks)**  
   - Ensure you can generate the merged DataFrame containing:  
     - OHLCV data  
     - Technical indicators  
     - Macro indicators  
   - Focus on getting **RSI** computed using **Code Snippets 8 and 9**.  
   - This process is essential and will help during the capstone project.

2. **Alternative (if tech indicators fail to generate):**  
   Download precomputed data using this snippet:

   ```python
   import gdown
   import pandas as pd

   file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
   gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
   df = pd.read_parquet("data.parquet", engine="pyarrow")

3. **RSI Strategy Setup:**  
   - RSI is already available in the dataset as a field.  
   - The threshold for **oversold** is defined as `RSI < 25`.

4. **Filter the dataset by RSI and date:**  
   ```python
   rsi_threshold = 25
   selected_df = df[
       (df['rsi'] < rsi_threshold) &
       (df['Date'] >= '2000-01-01') &
       (df['Date'] <= '2025-06-01')
   ]
5. **Calculate Net Profit Over 25 Years:**  
   - Total number of trades: **1568**  
   - For each trade, you invest **$1000**  
   - Use the 30-day forward return (`growth_future_30d`) to compute net earnings:  
     ```python
     net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
     ```

   - **Final Answer:**  
     What is the **net income in $K** (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?

In [34]:
import gdown

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")


Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=757e81c1-ae3b-4d20-950d-963e50e4ecdf
To: /content/data.parquet
100%|██████████| 130M/130M [00:02<00:00, 56.7MB/s]


In [37]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
net_income

np.float64(24295.523125248386)

---
### Q5. [Exploratory, Optional] Predicting a Positive-Return IPO

Most of the strategies for investing in IPOs deliver **negative average and median returns** (and even 75% quantiles).

**Question:**  
How would you change the strategy if you want to **increase the profitability**?

> This is an open-ended brainstorming question — propose ideas for identifying IPOs with positive future returns or building a more effective trading strategy.